In [2]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [3]:
import pandas as pd

# Replace 'your_file_path' with the actual path to your CSV file
df = pd.read_csv('ev-winners.csv')

# This will display the first few rows of the DataFrame
print(df.head())


   #                                               name batch date_announced  \
0  1                                  Tymofiy Mylovanov     1     2018-11-07   
1  2                                        Topos House     1     2018-11-07   
2  3                      18-year-old economics prodigy     1     2018-11-07   
3  4  Mark Lutter and his Center for Innovative Gove...     1     2018-11-07   
4  5                                     Harshita Arora     1     2018-11-07   

                                                link  \
0  marginalrevolution.com/marginalrevolution/2018...   
1  marginalrevolution.com/marginalrevolution/2018...   
2  marginalrevolution.com/marginalrevolution/2018...   
3  marginalrevolution.com/marginalrevolution/2018...   
4  marginalrevolution.com/marginalrevolution/2018...   

                                         description              type  \
0  Anonymous grant for writing in Eastern Europe....  Writing (Online)   
1  Pledged grant to San Francisco’

In [7]:
names = df['name'].to_numpy()
descriptions = df['description'].to_numpy()
# generate embeddings of the descriptions
embeddings = model.encode(descriptions)

In [9]:
import numpy as np

def get_ev_winners(query):

    #embeddings of the query, same dimensions as the description embeddings
    query_embed = np.tile(model.encode(query), (len(embeddings), 1))
    
    cos_sim = util.cos_sim(embeddings, query_embed)
    cos_sim
    
    # Add all pairs to a list with their cosine similarity score
    all_sentence_combinations = []
    for i in range(len(cos_sim)-1):
        all_sentence_combinations.append([cos_sim[i], descriptions[i], names[i]])
    
    # Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0][0], reverse=True)
    
    return all_sentence_combinations

In [11]:
def search_ev_winners(query, number):
    print(f"Top {number} matches for query: {query}\n")
    for index, item in enumerate(get_ev_winners(query)[:number], start=1):
        print(f"{index}. {item[2]}: {item[1]}")

search_ev_winners("book", 10)

Top 10 matches for query: book

1. Andy Matuschak: San Francisco, to support his project to reexamine and fundamentally improve the book as a method for learning and absorbing ideas, Twitter here. Here is his essay on why books do not work.
2. Kathleen Harward: Kathleen Harward, to write and market a series of children’s books based on classical liberal values.
3. Sonja Trauss: Sonja Trauss of YIMBY, assistance to publish Nicholas Barbon, A Defence of the Builder.
4. Marc Sidwell: Marc Sidwell of the United Kingdom, to write a book on common sense.
5. Jeffrey C. Huber: Jeffrey C. Huber, to write a book on tech and economic progress from a Christian point of view.
6. Joe Francis: Joe Francis, a farmer in Wales, to write a book on the economic and historical import of slavery in the American republic.
7. Tony Morley: Tony Morley, Progress Studies fellow, from Ngunnawal, Australia, to write the first optimistic children’s book on progress.
8. Henry Oliver: Henry Oliver, London, to write a